RQ1: What are the frames used by CNN and FOX in the stories using candidates’ tweets as news sources during the 2016 and 2020 US presidential campaign?
-	Data analysis of frames in news stories (percent, other statistics) 
-	Comparison between the 2 networks, and also comparison between 2016 and 2020 (CNN2016-2020, and Fox2016-2020)

Create a table like this
             		CNN2016 	FOX2016 	CNN2020 	FOX2020
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	-- include these 2, they are emotional frames
Negative 

Report differences, if any, CNN-FOX2016 and CNN-Fox2020, but also CNN2016-CNN2020, and FOX2016-FOX2020.


In [177]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
import pyreadstat
import statsmodels.api as sm

In [178]:
#open file
df=pd.read_csv('all.csv')
#cases STORYNUMBER 191 and 195 are removed since twitter had missing values

In [215]:
#unit of analysis is tweet
dftweet=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'NUMBEROFTWEETSBYCANDIDATES', 'NUMBEROFTWEETSFROMOTHERS', 'TWEETBY',
       'TWEETPRESENTATION', 'TWEETGENERICFRAMESDOMINANT','TWEETEMOTIONALFRAMES',
       'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE',
       'TWEETTOPIC', 'TWEETROLETOTHESTORY', 'STORYROLETOTWEET', 'SRC_TOTAL', 'STORYFACTCHECKSTWEET']]
#unit of analysis is story
dfstory=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'STORYTYPE', 'STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE',
       'STORYFACTCHECKSTWEET', 'SENSATIONALISM', 'src_expert', 'src_opponent',
       'src_otherpolitician', 'src_privatecitizen', 'src_media', 'src_other', "TWEETROLETOTHESTORY", "STORYROLETOTWEET", "SRC_TOTAL", "TWEETBY"]]
#remove duplicates
dfstory.drop_duplicates(subset=['STORYNUMBER'], keep='first', inplace=True)
#dv list for tweets and stories
DVtweet=['TWEETTOPIC','TWEETGENERICFRAMESDOMINANT', 'TWEETEMOTIONALFRAMES',
       #  'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE'
        ]
DVstory=['STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       # 'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE'
       ]
DVnamesTweet=['the topic of the tweet','the generic frames of the tweets','the emotional frames of the tweets']
DVnamesStory=['the topic of the story','the generic frames of the stories','the emotional frames of the stories']
#list of different datasets
dftweet2020=dftweet.loc[dftweet['YEAR'] == 2020]
dftweet2016=dftweet.loc[dftweet['YEAR'] == 2016]
dftweetFOX=dftweet.loc[dftweet['PUBLISHER'] == "fox"]
dftweetCNN=dftweet.loc[dftweet['PUBLISHER'] == "cnn"]
dftweetFOX2016=dftweetFOX.loc[dftweetFOX['YEAR'] == 2016]
dftweetCNN2016=dftweetCNN.loc[dftweetCNN['YEAR'] == 2016]
dftweetFOX2020=dftweetFOX.loc[dftweetFOX['YEAR'] == 2020]
dftweetCNN2020=dftweetCNN.loc[dftweetCNN['YEAR'] == 2020]
dfstory2020=dfstory.loc[dfstory['YEAR'] == 2020]
dfstory2016=dfstory.loc[dfstory['YEAR'] == 2016]
dfstoryFOX=dfstory.loc[dfstory['PUBLISHER'] == "fox"]
dfstoryCNN=dfstory.loc[dfstory['PUBLISHER'] == "cnn"]
dfstoryFOX2016=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2016]
dfstoryCNN2016=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2016]
dfstoryFOX2020=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2020]
dfstoryCNN2020=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2020]

/var/folders/_8/_r3pgsjx2l74qstm6wlz1m8r0000gr/T/ipykernel_72859/1834556026.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfstory.drop_duplicates(subset=['STORYNUMBER'], keep='first', inplace=True)


In [180]:
pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], margins=True)

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
STORYGENERICFRAMESDOMINANT,,,,,
conflict,55,91,15,56,217
economic,3,11,1,0,15
human interest,3,12,1,17,33
morality,1,0,1,0,2
responsibility,2,5,0,3,10
All,64,119,18,76,277


In [181]:
pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
STORYGENERICFRAMESDOMINANT,,,,
conflict,25.35,41.94,6.91,25.81
economic,20.00,73.33,6.67,0.00
human interest,9.09,36.36,3.03,51.52
morality,50.00,0.00,50.00,0.00
responsibility,20.00,50.00,0.00,30.00
All,23.10,42.96,6.50,27.44


In [182]:
pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], margins=True)

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
STORYEMOTIONALFRAMES,,,,,
3,3,0,2,0,5
negative,60,108,16,55,239
positive,1,11,1,21,34
All,64,119,19,76,278


In [183]:
pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
STORYEMOTIONALFRAMES,,,,
3,60.00,0.00,40.00,0.00
negative,25.10,45.19,6.69,23.01
positive,2.94,32.35,2.94,61.76
All,23.02,42.81,6.83,27.34


RQ2: What are the frames of the candidates’ tweets selected by journalists for inclusion in news stories?
-	Data analysis of frames in candidates tweets
-	Presentation of tweets in news stories 
-	2016 and 2020

Create a table like this
             	Trump2016	Pence	Hillary  Kaine Trump2020   Pence2020   Biden    Harris
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	
Negative


In [184]:
tdf1=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], margins=True)
tdf1

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
TWEETGENERICFRAMESDOMINANT,,,,,
conflict,94,73,20,73,260
economic,1,16,1,3,21
human interest,14,36,13,32,95
morality,4,9,0,3,16
responsibility,10,7,0,9,26
All,123,141,34,120,418


In [185]:
tdf2=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
tdf2

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
TWEETGENERICFRAMESDOMINANT,,,,
conflict,36.15,28.08,7.69,28.08
economic,4.76,76.19,4.76,14.29
human interest,14.74,37.89,13.68,33.68
morality,25.00,56.25,0.00,18.75
responsibility,38.46,26.92,0.00,34.62
All,29.43,33.73,8.13,28.71


In [186]:
tdf3=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], margins=True)
tdf3

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
TWEETEMOTIONALFRAMES,,,,,
3,5,0,2,0,7
negative,104,84,20,72,280
positive,14,58,12,48,132
All,123,142,34,120,419


In [187]:
tdf4=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100 
tdf4

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
TWEETEMOTIONALFRAMES,,,,
3,71.43,0.00,28.57,0.00
negative,37.14,30.00,7.14,25.71
positive,10.61,43.94,9.09,36.36
All,29.36,33.89,8.11,28.64


In [188]:
tdf1.to_csv('RQ2_1.csv')
tdf2.to_csv('RQ2_2.csv')
tdf3.to_csv('RQ2_3.csv')
tdf4.to_csv('RQ2_4.csv')

RQ3: How did CNN.com and FoxNews.com report Donald Trump’s tweets compared to the other candidates in the 2016 and 2020 presidential election campaign news stories? Are there any significant differences between the two campaign coverages?
-	Report that the majority of news stories using tweets were using Trump’s tweets
-	Report presentation of tweets (embedding, citing, etc.)
-	Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2
-	Report fact-checking, sensationalism, number of sources, and role of story to tweets for Trump vs Hillary and Biden. 


In [209]:
# Report that the majority of news stories using tweets were using Trump’s tweets
pd.crosstab(dfstory['TWEETBY'], dfstory['PUBLISHERYEAR'], margins=True)

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
TWEETBY,,,,,
biden,10,5,1,6,22
harris,0,6,0,4,10
pence,3,1,2,0,6
trump,51,107,16,66,240
All,64,119,19,76,278


In [213]:
# Report presentation of tweets (embedding, citing, etc.)
rq3_1=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_1p=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq3_2=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_2p=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq3_1.to_csv('RQ3_4.csv')
rq3_1p.to_csv('RQ3_4p.csv')
rq3_2.to_csv('RQ3_5.csv')
rq3_2p.to_csv('RQ3_5p.csv')

In [ ]:
# Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2

In [218]:
# Report fact-checking, sensationalism, number of sources, and role of story to tweets for Trump vs Hillary and Biden.
rq3_3=pd.crosstab(dftweet['STORYFACTCHECKSTWEET'], dftweet['TWEETBY'], margins=True)
rq3_3p=pd.crosstab(dftweet['STORYFACTCHECKSTWEET'], dftweet['TWEETBY'], margins=True, normalize='index').round(4)*100
rq3_4=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['TWEETBY'], margins=True)
rq3_4p=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['TWEETBY'], margins=True, normalize='index').round(4)*100
# rq3_5=pd.crosstab(dfstory['SRC_TOTAL'], dfstory['TWEETBY'], margins=True)
# rq3_5p=pd.crosstab(dfstory['SRC_TOTAL'], dfstory['TWEETBY'], margins=True, normalize='index').round(4)*100
rq3_6=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['TWEETBY'], margins=True)
rq3_6p=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['TWEETBY'], margins=True, normalize='index').round(4)*100
rq3_7=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['TWEETBY'], margins=True)
rq3_7p=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['TWEETBY'], margins=True, normalize='index').round(4)*100
rq3_3.to_csv('RQ3_1.csv')
rq3_3p.to_csv('RQ3_1p.csv')
rq3_4.to_csv('RQ3_2.csv')
rq3_4p.to_csv('RQ3_2p.csv')
# rq3_5.to_csv('RQ3_3.csv')
# rq3_5p.to_csv('RQ3_3p.csv')
rq3_6.to_csv('RQ3_6.csv')
rq3_6p.to_csv('RQ3_6p.csv')
rq3_7.to_csv('RQ3_7.csv')
rq3_7p.to_csv('RQ3_7p.csv')

In [219]:
rq3_3

TWEETBY,biden,harris,pence,trump,All
STORYFACTCHECKSTWEET,,,,,
no,41,14,7,240,302
yes,3,0,1,112,116
All,44,14,8,352,418


In [220]:
rq3_4

TWEETBY,biden,harris,pence,trump,All
SENSATIONALISM,,,,,
no,17,9,6,211,243
yes,5,1,0,29,35
All,22,10,6,240,278


In [222]:
rq3_6

TWEETBY,biden,harris,pence,trump,All
TWEETROLETOTHESTORY,,,,,
illustration,7,1,5,89,102
non-primary source,8,2,1,49,60
trigger,7,7,0,102,116
All,22,10,6,240,278


In [223]:
rq3_7

TWEETBY,biden,harris,pence,trump,All
STORYROLETOTWEET,,,,,
amplifies the tweet,5,3,0,53,61
ctiticizes the tweet,3,2,0,93,98
neutral,14,5,6,94,119
All,22,10,6,240,278


In [228]:
pg.anova(data=dfstory2020, dv='SRC_TOTAL', between='TWEETBY', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,TWEETBY,2.154611,3,0.718204,0.563315,0.639849,0.00877
1,Within,243.517183,191,1.274959,NaN,NaN,NaN


In [227]:
pg.pairwise_ttests(data=dfstory2020, dv='SRC_TOTAL', between='TWEETBY', padjust='bonf', effsize='cohen')

/Users/stephen.w.song/opt/anaconda3/envs/mystats/lib/python3.11/site-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
/Users/stephen.w.song/opt/anaconda3/envs/mystats/lib/python3.11/site-packages/pingouin/parametric.py:257: RuntimeWarning: Degrees of freedom <= 0 for slice
  vx, vy = x.var(ddof=1), y.var(ddof=1)
/Users/stephen.w.song/opt/anaconda3/envs/mystats/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/stephen.w.song/opt/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/stephen.w.song/opt/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_stats_py.py:110

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,TWEETBY,biden,harris,False,True,1.073663,17.282637,two-sided,0.297736,1.000000,bonf,0.587,0.459631
1,TWEETBY,biden,pence,False,True,2.318694,10.000000,two-sided,0.042868,0.214340,bonf,1.92,0.699113
2,TWEETBY,biden,trump,False,True,0.335141,10.984553,two-sided,0.743832,1.000000,bonf,0.319,0.117857
3,TWEETBY,harris,pence,False,True,1.500000,9.000000,two-sided,0.167851,0.839253,bonf,0.734,0.474342
4,TWEETBY,harris,trump,False,True,-1.336787,10.964501,two-sided,0.208368,1.000000,bonf,0.634,-0.334758
5,TWEETBY,pence,trump,False,True,NaN,1.000000,two-sided,NaN,NaN,bonf,nan,NaN


RQ5: What journalistic values were used by CNN and FOX in the stories using candidates’ tweets as news stories?
-	Sensationalism.  (percent, other statistics, comparison between the 2 media organizations)
-	Fact-checking
-	Number of sources 
-	Role of story to tweets
-	Other values we coded for

In [189]:
rq5_1=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_1

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
STORYCLAIMSANOTHERSOURCEISFALSE,,,,,
no,42,77,17,55,191
yes,22,42,2,21,87
All,64,119,19,76,278


In [196]:
rq5_4p=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq5_4p

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
TWEETROLETOTHESTORY,,,,
illustration,33.33,35.29,7.84,23.53
non-primary source,25.00,55.00,15.00,5.00
trigger,12.93,43.10,1.72,42.24
All,23.02,42.81,6.83,27.34


In [190]:
rq5_1p=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq5_1p

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
STORYCLAIMSANOTHERSOURCEISFALSE,,,,
no,21.99,40.31,8.90,28.80
yes,25.29,48.28,2.30,24.14
All,23.02,42.81,6.83,27.34


In [191]:
rq5_2=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_2

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
STORYFACTCHECKSTWEET,,,,,
no,43,80,18,68,209
yes,21,39,1,8,69
All,64,119,19,76,278


In [192]:
rq5_2p=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq5_2p

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
STORYFACTCHECKSTWEET,,,,
no,20.57,38.28,8.61,32.54
yes,30.43,56.52,1.45,11.59
All,23.02,42.81,6.83,27.34


In [193]:
rq5_3=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_3

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020,All
SENSATIONALISM,,,,,
no,51,117,17,58,243
yes,13,2,2,18,35
All,64,119,19,76,278


In [194]:
rq5_3p=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], margins=True, normalize='index').round(4)*100
rq5_3p

PUBLISHERYEAR,cnn2016,cnn2020,fox2016,fox2020
SENSATIONALISM,,,,
no,20.99,48.15,7.00,23.87
yes,37.14,5.71,5.71,51.43
All,23.02,42.81,6.83,27.34


In [202]:
#comparison of number of sources: Significant difference between CNN and FOX in 2020 (no data for 2016)
pg.ttest(dfstoryCNN2020.SRC_TOTAL, dfstoryFOX2020.SRC_TOTAL, paired=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.040818,166.75265,two-sided,0.042846,"[-0.65, -0.01]",0.295797,1.096,0.517768


In [205]:
dfstoryCNN2020.describe().round(2)

,STORYNUMBER,CODER,YEAR,SRC_TOTAL
count,119.0,119.00,119.0,119.00
mean,125.7,1.87,2020.0,1.63
std,46.9,0.34,0.0,1.14
min,50.0,1.00,2020.0,0.00
25%,79.5,2.00,2020.0,1.00
50%,137.0,2.00,2020.0,2.00
75%,166.5,2.00,2020.0,2.00
max,198.0,2.00,2020.0,4.00


In [206]:
dfstoryFOX2020.describe().round(2)

,STORYNUMBER,CODER,YEAR,SRC_TOTAL
count,76.00,76.00,76.0,76.00
mean,56.04,1.36,2020.0,1.96
std,43.91,0.48,0.0,1.08
min,1.00,1.00,2020.0,0.00
25%,19.75,1.00,2020.0,1.00
50%,38.50,1.00,2020.0,2.00
75%,106.25,2.00,2020.0,3.00
max,130.00,2.00,2020.0,5.00
